In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
  "What is the distance between {country_a} and {country_b}"
)
# prompt = template.format(country_a="Mexico", country_b="Thailand")
# chat.predict(prompt)

messages = ChatPromptTemplate.from_messages([
  ("system", "You are geography export. And you only reply in {language}."),
  ("ai", "Ciao, mi chiamo {name}!"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = messages.format_messages(
  language="Greek",
  name="Plato",
  country_a="South Korea",
  country_b="Greece"
)

chat.predict_messages(prompt)

### BaseOutputParser


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
  def parse(self, text):
    item = text.strip().split(",")
    return list(map(str.strip, item))
  
p = CommaOutputParser()
p.parse("Hello, how, are, you")

template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else"),
  ("human", "{question}")
])

prompt = template.format_messages(
  max_items=10,
  question="What are the colors"
)

result = chat.predict_messages(prompt)
p.parse(result.content)

### LangChain Expression Language(LCEL) : Chain


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
  def parse(self, text):
    item = text.strip().split(",")
    return list(map(str.strip, item))

template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else"),
  ("human", "{question}")
])

# 입력 => chat model => outputparser 순으로 실행
chain = template | chat | CommaOutputParser()

chain.invoke({
  "max_items":5,
  "question": "What are the pokemons? please reply in korean"
})


### LCEL : Chain과 Chain 연결


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
  ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef specilaized on making traditional recipes vegerarian. You find altenative ingredients and explain their preparation. You don't radically modify the recipe. If there in no alternative for a food just say you don't know how to replace it"),
  ("human", "{recipe}")
])

# Runnable Map
veg_chain = veg_chef_prompt | chat

final_chains = {"recipe":chef_chain} | veg_chain
final_chains.invoke({
  "cuisine" : "French"
})

### FewShotPromptTemplate


In [ ]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
        {
            "question": "France",
            "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro""",
        },
        {"question": "Italy",
            "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """
        },
        {
            "question": "Greece",
            "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """
        }
    ]

# example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")
# prompt = FewShotPromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples,
#     suffix="Human: What do you know about {country}",
#     input_variables=["country"]
# )

# prompt.format(country="Germany")

# chain = prompt | chat
# chain.invoke({
#     "country": "Germany"
# })

class RnadomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples

  def add_example(self, example):
    self.examples.append(example)
    

  def select_examples(self, input_variables):
    from random import choice
    return [choice(self.examples)]
    

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {question}?"),
  ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

example_selector = LengthBasedExampleSelector(
  examples=examples,
  example_prompt=example_prompt,
  max_length=10
)

final_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a geography export"),
  example_selector,
  ("human", "What do you know about {question}?")
])

chain = final_prompt | chat

chain.invoke({"question": "Germany"})


### Load prompt, Composite prompts


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, load_prompt
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

# prompt = load_prompt("./prompt.yaml")
# prompt.format(country="Germany")

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]


full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# full_prompt.format(
#   character="Pirate",
#   example_question="What is your location",
#   example_answer="Arrrg! That is secret!! Arg arg!!",
#   question="What is your favorite food?"
# )

chain = full_prompt | chat
chain.invoke({
  "character":"Pirate",
  "example_question":"What is your location",
  "example_answer":"Arrrg! That is secret!! Arg arg!!",
  "question":"What is your favorite food?"
})

### Caching


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
# set_debug(True)

chat = ChatOpenAI(temperature=0.1)

chat.predict("How do you make italian food?")

### Usage : calculate usage cost


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(temperature=0.1)

with get_openai_callback() as usage:
  a = chat.predict("What is the recipe for soju?")
  b = chat.predict("What is the recipe for Galbi?")
  print(a,b, "\n")
  print(usage)